## imports

In [2]:
%matplotlib inline
import utils; reload(utils)
from utils import *

In [20]:
import numpy as np
import dicom
import glob
from matplotlib import pyplot as plt
import os
import cv2
#import mxnet as mx
import pandas as pd
from sklearn import cross_validation
import xgboost as xgb
from keras.utils.np_utils import to_categorical

In [7]:
def get_3d_data(path):
    slices = [dicom.read_file(path + '/' + s) for s in os.listdir(path)]
    slices.sort(key=lambda x: int(x.InstanceNumber))
    return np.stack([s.pixel_array for s in slices])

In [8]:
def get_data_id(path,source='npy'): #source can be the standard folder, or an npy file
    
    if source=='folder':
        sample_image = get_3d_data(path)
        sample_image[sample_image == -2000] = 0
    elif source=='npy':
        sample_image=np.load(path)
    # f, plots = plt.subplots(4, 5, sharex='col', sharey='row', figsize=(10, 8))

    batch = []
    cnt = 0
    dx = 40
    ds = 512
    for i in range(0, sample_image.shape[0] - 3, 3):
        tmp = []
        for j in range(3):
            img = sample_image[i + j]
            img = 255.0 / np.amax(img) * img
            img = cv2.equalizeHist(img.astype(np.uint8))
            img = img[dx: ds - dx, dx: ds - dx]
            img = cv2.resize(img, (224, 224))
            tmp.append(img)

        tmp = np.array(tmp)
        batch.append(np.array(tmp))

        # if cnt < 20:
        #     plots[cnt // 5, cnt % 5].axis('off')
        #     plots[cnt // 5, cnt % 5].imshow(np.swapaxes(tmp, 0, 2))
        # cnt += 1

    # plt.show()
    batch = np.array(batch)
    return batch

## preprocessing

In [3]:
path='/mnt/kaggle_lung_cancer//stage1'
path_np='/mnt/kaggle_lung_cancer//stage1_segmented'

In [ ]:
os.listdir(path)

In [5]:
sample_image=get_3d_data(path+'/c02daa2116b27ce9b023fe7809ef5be9')
sample_image[sample_image == -2000] = 0
sample_image.shape

NameError: name 'get_3d_data' is not defined

In [6]:
im=np.load(path_np+'/c02daa2116b27ce9b023fe7809ef5be9'+'.npy')
im.shape

IOError: [Errno 2] No such file or directory: '/mnt/kaggle_lung_cancer//stage1_segmented/c02daa2116b27ce9b023fe7809ef5be9.npy'

In [ ]:
#get all data
all_batches=[]
for folder in glob.glob('/mnt/kaggle_lung_cancer/stage1/*'):
    print folder
    patient_id=folder.split('/')[-1]
    id_full_list.append(patient_id)
    batch = get_data_id(folder)
    print batch.shape
    all_batches.append(batch)

In [11]:
all_batches

[]

### fine tuning vgg

In [71]:
from vgg16bn import Vgg16BN
vgg = Vgg16BN()
model = vgg.model

/home/ubuntu/.local/lib/python2.7/site-packages/keras/layers/core.py:621: UserWarning: `output_shape` argument not specified for layer lambda_2 and cannot be automatically inferred with the Theano backend. Defaulting to output shape `(None, 3, 224, 224)` (same as input shape). If the expected output shape is different, specify it via the `output_shape` argument.
  .format(self.name, input_shape))


In [72]:
## finetune vgg - split layers
layers = model.layers
last_conv_idx = [index for index,layer in enumerate(layers) if type(layer) is Convolution2D][-1]
conv_layers = layers[:last_conv_idx+1]
conv_model = Sequential(conv_layers)
# Dense layers - also known as fully connected or 'FC' layers
fc_layers = layers[last_conv_idx+1:]

In [81]:
# predict the batches
conv_features_list=[]
for num,batch in enumerate(all_batches):
    conv_features = conv_model.predict(batch, batch_size=4)
    conv_features_list.append(conv_features)
    print num
conv_features_array=np.array(conv_features_list)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

In [87]:
np.save('/datadrive/kaggle_ds_bowl_17/vgg_weights/vgg_conv_feats.npy',conv_features_array)

In [93]:
#do mean of batches
conv_features_mean=[]
for i in conv_features_array:
    conv_features_mean.append(np.mean(i,axis=0))
conv_features_mean_array=np.array(conv_features_mean)

In [117]:
full_train_codes=[i in df['id'].values for i in id_full_list]

In [149]:
#top layers
def get_bn_layers(p):
    return [
        MaxPooling2D(input_shape=conv_layers[-1].output_shape[1:]),
        BatchNormalization(axis=1),
        Dropout(p/4),
        Flatten(),
        Dense(512, activation='relu'),
        BatchNormalization(),
        Dropout(p),
        Dense(512, activation='relu'),
        BatchNormalization(),
        Dropout(p/2),
        Dense(2, activation='softmax')
    ]
bn_model = Sequential(get_bn_layers(0.6))
bn_model.compile(Adam(lr=0.001), loss='categorical_crossentropy', metrics=['accuracy'])

In [120]:
#generate as always true/false list of train batchs. some work whould be done here
df = pd.read_csv('/datadrive/kaggle_ds_bowl_17/stage1_labels.csv')
x=conv_features_mean_array[full_train_codes]
y = df['cancer'].as_matrix()
y.shape,x.shape
cat_y=to_categorical(y)

((1397,), (1397, 512, 14, 14))

In [135]:
#split train test
trn_features, val_features, trn_labels, val_labels = cross_validation.train_test_split(x, cat_y, random_state=42, stratify=y,test_size=0.20)

In [159]:
#train
bn_model.fit(trn_features, trn_labels, nb_epoch=4, batch_size=32, validation_data=(val_features, val_labels))

Train on 1117 samples, validate on 280 samples
Epoch 1/4
1117/1117 [==============================] - 0s - loss: 1.0122 - acc: 0.5560 - val_loss: 5.1540 - val_acc: 0.2821

In [160]:
bn_model.optimizer.lr=1e-4
bn_model.fit(trn_features, trn_labels, nb_epoch=4, batch_size=32, validation_data=(val_features, val_labels))

Train on 1117 samples, validate on 280 samples
Epoch 1/4
1117/1117 [==============================] - 0s - loss: 0.5450 - acc: 0.7601 - val_loss: 0.9071 - val_acc: 0.7179

In [46]:
#predict the vgg on the image gropus/batches without any fintuning
source=glob.glob('/datadrive/kaggle_ds_bowl_17/stage1/*') #folders
source=glob.glob('/datadrive/kaggle_ds_bowl_17/stage1_segmented/*') #npys segmented
for folder in source:
    print folder
    patient_id=folder.split('/')[-1]
    batch = get_data_id(folder)
    print batch.shape
    trn_features = model.predict(batch, batch_size=4)
    #np.save()
    np.save('/datadrive/kaggle_ds_bowl_17/vgg_weights//'+patient_id+'.npy', trn_features)

/datadrive/kaggle_ds_bowl_17/stage1/c02daa2116b27ce9b023fe7809ef5be9
(40, 3, 224, 224)
/datadrive/kaggle_ds_bowl_17/stage1/eb9db3f740f8e153e85f83c57bc4e522
(83, 3, 224, 224)
/datadrive/kaggle_ds_bowl_17/stage1/f39dacd5f134f3f4482094d3f8a0aff1
(79, 3, 224, 224)
/datadrive/kaggle_ds_bowl_17/stage1/504e447ad62ea9ebb283873e044b5dd2
(53, 3, 224, 224)
/datadrive/kaggle_ds_bowl_17/stage1/8b494d14d835dd5ae13dab19b9520a55
(36, 3, 224, 224)
/datadrive/kaggle_ds_bowl_17/stage1/402a18fa05523f80abb1e6d97793cc19
(48, 3, 224, 224)
/datadrive/kaggle_ds_bowl_17/stage1/e3e518324e1a85b85f15d9127ed9ea89
(57, 3, 224, 224)
/datadrive/kaggle_ds_bowl_17/stage1/39dc5f09d7b8eeda7f2f02c86f53247d
(65, 3, 224, 224)
/datadrive/kaggle_ds_bowl_17/stage1/48ab0b98fc7789304c21430978624f32
(50, 3, 224, 224)
/datadrive/kaggle_ds_bowl_17/stage1/7b43882369098df840e51421e6059aea
(110, 3, 224, 224)
/datadrive/kaggle_ds_bowl_17/stage1/6828335aec9aa0d0053ec0949319aa4b
(42, 3, 224, 224)
/datadrive/kaggle_ds_bowl_17/stage1/ad5f3a

### resnet

In [18]:
from keras.applications.resnet50 import ResNet50
from keras.applications.inception_v3 import InceptionV3

In [ ]:
model_incept=inception_v3(include_top=False)

In [19]:
model_resnet=ResNet50(include_top=False)

In [12]:
#mess with layers
layers = model.layers
last_conv_idx = [index for index,layer in enumerate(layers)  if type(layer) is Convolution2D][-1]
conv_layers = layers[:last_conv_idx+1]
x=conv_layers[0]
conv_model = Sequential(conv_layers)
# Dense layers - also known as fully connected or 'FC' layers
fc_layers = layers[last_conv_idx+1:]

In [55]:
#predict the resnet on the image gropus/batches without any fintuning
source=glob.glob('/datadrive/kaggle_ds_bowl_17/stage1/*') #folders
source=glob.glob('/datadrive/kaggle_ds_bowl_17/stage1_segmented/*') #npys segmented
for folder in source:
    if folder in files:
        print 'skipped'
    else:
        try:
            print folder

            patient_id=folder.split('/')[-1]
            batch = get_data_id(folder)
            print batch.shape
            trn_features = model.predict(batch, batch_size=4)
            #np.save()
            np.save('/datadrive/kaggle_ds_bowl_17/resnet_keras_weights//'+patient_id, trn_features)
        except:
            pass

/datadrive/kaggle_ds_bowl_17/stage1_segmented/5f383eb9c3d8ea72ddec7e2e874d577d.npy
(57, 3, 224, 224)
/datadrive/kaggle_ds_bowl_17/stage1_segmented/502e157e6a22166997d054e8920d921c.npy
(54, 3, 224, 224)
/datadrive/kaggle_ds_bowl_17/stage1_segmented/01de8323fa065a8963533c4a86f2f6c1.npy
(76, 3, 224, 224)
/datadrive/kaggle_ds_bowl_17/stage1_segmented/fa9575f64e6881c6b2730f0e225c9573.npy
(42, 3, 224, 224)
/datadrive/kaggle_ds_bowl_17/stage1_segmented/b8793dbd40de88c0de0913abbaab0fe7.npy
(85, 3, 224, 224)
/datadrive/kaggle_ds_bowl_17/stage1_segmented/5791c42d317f34592be9a933c50e68ad.npy
(43, 3, 224, 224)
/datadrive/kaggle_ds_bowl_17/stage1_segmented/a590753c175011e42bf17b90e3aec3b0.npy
(34, 3, 224, 224)
/datadrive/kaggle_ds_bowl_17/stage1_segmented/e314fd13809db0132443b924401d828b.npy
(52, 3, 224, 224)
/datadrive/kaggle_ds_bowl_17/stage1_segmented/6d60c26d7d35ffd967127b380ff01c12.npy
(58, 3, 224, 224)
/datadrive/kaggle_ds_bowl_17/stage1_segmented/c67e799bcc1e2635eb9164f6e8cf75f3.npy
(172, 3,

### xgboost

In [ ]:
# Best iteration:
# on resnet: [50]	validation_0-logloss:0.558415

In [ ]:
#ive saved all ouputs of vgg. now:
# 1 load them V
# 2 mean them V
# 3 build linreg on them V
# 4 build smth else on them like xgboost V
# 5 do the same for resnet
####
# 1 fine tune vgg
# 2 fine tune resnet
# 3 do all the above withsmmetric batches

In [77]:
df = pd.read_csv('/datadrive/kaggle_ds_bowl_17/stage1_labels.csv')

In [38]:
%cd /datadrive/kaggle_ds_bowl_17/resnet_keras_weights/

/datadrive/kaggle_ds_bowl_17/resnet_keras_weights


In [52]:
g=glob.glob('*.npy')

In [53]:
files=[file.split('.')[0] for file in g]
len(files)

871

In [78]:
#this 1 is missing
df=df[df['id']!='e537c91cdfa97d20a39df7ef04a52570']

In [61]:
#x = np.array(np.mean(all_batches, axis=0))
y = df['cancer'].as_matrix()

In [79]:
#read and mean. it would take a few mins to do
x = np.array([np.mean(np.load('/datadrive/kaggle_ds_bowl_17/resnet_keras_weights//%s.npy' % str(id)), axis=0) for id in df['id'].tolist()])

In [80]:
x.shape

(1396, 1000)

In [63]:
trn_x, val_x, trn_y, val_y = cross_validation.train_test_split(x, y, random_state=42, stratify=y,test_size=0.20)

In [64]:
#fed vgg straight to xgboost
clf = xgb.XGBRegressor(max_depth=10, n_estimators=1500,min_child_weight=9,learning_rate=0.05,
                       nthread=8, subsample=0.80,colsample_bytree=0.80,seed=4242)

clf.fit(trn_x, trn_y, eval_set=[(val_x, val_y)], verbose=True, early_stopping_rounds=50)



[0]	validation_0-logloss:0.680403
Will train until validation_0-logloss hasn't improved in 50 rounds.
[1]	validation_0-logloss:0.669837
[2]	validation_0-logloss:0.661146
[3]	validation_0-logloss:0.651974
[4]	validation_0-logloss:0.646499
[5]	validation_0-logloss:0.639838
[6]	validation_0-logloss:0.6347
[7]	validation_0-logloss:0.626201
[8]	validation_0-logloss:0.617863
[9]	validation_0-logloss:0.613002
[10]	validation_0-logloss:0.607287
[11]	validation_0-logloss:0.60234
[12]	validation_0-logloss:0.59784
[13]	validation_0-logloss:0.595506
[14]	validation_0-logloss:0.592061
[15]	validation_0-logloss:0.58918
[16]	validation_0-logloss:0.585081
[17]	validation_0-logloss:0.582275
[18]	validation_0-logloss:0.582173
[19]	validation_0-logloss:0.580003
[20]	validation_0-logloss:0.577068
[21]	validation_0-logloss:0.574667
[22]	validation_0-logloss:0.573972
[23]	validation_0-logloss:0.573306
[24]	validation_0-logloss:0.571263
[25]	validation_0-logloss:0.570319
[26]	validation_0-logloss:0.568203
[2

XGBRegressor(base_score=0.5, colsample_bylevel=1, colsample_bytree=0.8,
       gamma=0, learning_rate=0.05, max_delta_step=0, max_depth=10,
       min_child_weight=9, missing=None, n_estimators=1500, nthread=8,
       objective='reg:linear', reg_alpha=0, reg_lambda=1,
       scale_pos_weight=1, seed=4242, silent=True, subsample=0.8)

###  submit

In [75]:
x.shape

(198, 1000)

In [83]:
clf.fit(x, y, verbose=True, eval_metric='logloss')

XGBRegressor(base_score=0.5, colsample_bylevel=1, colsample_bytree=0.8,
       gamma=0, learning_rate=0.05, max_delta_step=0, max_depth=10,
       min_child_weight=9, missing=None, n_estimators=1500, nthread=8,
       objective='reg:linear', reg_alpha=0, reg_lambda=1,
       scale_pos_weight=1, seed=4242, silent=True, subsample=0.8)

In [68]:
%cd ~/Notebooks/kaggle_ds_bowl_17/

/home/ubuntu/Notebooks/kaggle_ds_bowl_17


In [84]:
##### make submsision
#vgg+xgboost: val: 0.57 lb:0.6
#i had .558...
#[53]	validation_0-logloss:0.559103
df = pd.read_csv('/datadrive/kaggle_ds_bowl_17/stage1_sample_submission.csv')

x = np.array([np.mean(np.load('/datadrive/kaggle_ds_bowl_17/resnet_keras_weights//%s.npy' % str(id)), axis=0) for id in df['id'].tolist()])

pred = clf.predict(x)
pred[pred<0.02]=0.02
pred[pred>0.98]=0.98

df['cancer'] = pred
df.to_csv('subm_resnet1_mean_xgb_segmented.csv', index=False)
print(df.head())

                                 id    cancer
0  026470d51482c93efc18b9803159c960  0.231427
1  031b7ec4fe96a3b035a8196264a8c8c3  0.630414
2  03bd22ed5858039af223c04993e9eb22  0.355199
3  06a90409e4fcea3e634748b967993531  0.238387
4  07b1defcfae5873ee1f03c90255eb170  0.313967
